In [2]:
import cv2
from matplotlib import pyplot as plt

# Load an example image
image = cv2.imread("your_image.jpg")  # Replace "your_image.jpg" with an actual image file path
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

# Display the image
plt.imshow(image)
plt.axis("off")
plt.show()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
